# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | companions.ipynb
 |-data\Orcus - Companions.xlsx
 |-output
```

and will take data\Orcus - Companions.xlsx and create a markdown file output\companions.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Companions.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Chapter,Name,Alignment,Size,Ancestry,Type,Tag,Level,Role,Senses,...,Language,Skills,Str,Con,Dex,Int,Wis,Cha,Equipment,Description
0,a,a,a,a,a,a,a,a,a,a,...,a,a,a,a,a,a,a,a,a,a
1,Cup of Brimstone,Chapter,,,,,,,,,...,,,,,,,,,,
2,Cup of Brimstone,Demon Toad,Chaotic evil,Small,Outsider,Monstrosity,Demon,1,Summoned Creature,darkvision,...,,,,,,,,,,
3,Cup of Brimstone,Hopping Imp,Chaotic evil,Small,Outsider,Humanoid,Demon,1,Summoned Creature,darkvision,...,,,,,,,,,,
4,Cup of Brimstone,Burner Demon,Chaotic evil,Medium,Outsider,Humanoid,"Demon, Fire",5,Summoned Creature,darkvision,...,,,,,,,,,,


In [3]:
#cell 2, examine 5 random companions

data.sample(5)

,Chapter,Name,Alignment,Size,Ancestry,Type,Tag,Level,Role,Senses,...,Language,Skills,Str,Con,Dex,Int,Wis,Cha,Equipment,Description
1,Cup of Brimstone,Chapter,,,,,,,,,...,,,,,,,,,,
16,Animal Companions,Chapter,,,,,,,,,...,,,,,,,,,,
29,Animal Companions,Hunting Spider,Unaligned,Medium,Natural,Beast,,,Animal Companion,darkvision,...,,"Athletics +7 + half your level, Stealth +6 + h...",14,16,12,2,14,10,,A web-throwing spider. Could also be used for ...
18,Animal Companions,Arboreal Sapling,Unaligned,Small,Natural,Automaton,,,Animal Companion,low-light vision,...,,Nature +7 + half your level,16,12,14,2,14,10,,Could also be used for a wood golem.
27,Animal Companions,Horse,Unaligned,Large,Natural,Beast,,,Animal Companion,,...,,Endure +6 + half your level,16,12,14,2,14,10,,"A horse, camel or llama."


In [4]:
#cell 3
#we define a function which takes in a row of this power dataframe and returns an md string

def companion_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    if row_dict.get('Name') == 'Chapter':
        md = '# {Chapter}  '.format(**row_dict)
        if row['Description']:
            md += "\n{Description}  ".format(**row_dict)
        return md

    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
        
    #start with name. md is the string we'll be outputing
    md = """\n### {Name}  """.format(**row_dict)
    
    if row['Level']:
        md += "\n**Level {Level} {Role}**  ".format(**row_dict)
    else: 
        md += """\n**{Role}**  """.format(**row_dict)
    
    md += """\n{Size} {Ancestry} {Type}""".format(**row_dict)

    if row['Tag']:
        md += " ● {Tag}".format(**row_dict)

    if row['Alignment']:
        md += " ({Alignment})".format(**row_dict)
        
    md += """  """.format(**row_dict)

    if row['Senses']:
        if row['Skills']:
            md += """\n**Senses:** {Senses}; **Skills:** {Skills}  """.format(**row_dict)
        else: 
            md += """\n**Senses:** {Senses}  """.format(**row_dict) 
    elif row['Skills']:
        md += """\n**Skills:** {Skills}  """.format(**row_dict) 
    
    if row['Language']:
        md += """\n**Languages:** {Language}  """.format(**row_dict)

    if row['Str']:
        md += """\n**Str:** {Str}, **Con:** {Con}, **Dex:** {Dex}  \n**Int:** {Int}, **Wis:** {Wis}, **Cha:** {Cha}  """.format(**row_dict)
    
    if row['Equipment']:
        md += """\n**Equipment:** {Equipment}  """.format(**row_dict)
    
    md += """\n""".format(**row_dict)
    
    md += """\n**Speed:** {Spd}  """.format(**row_dict)
    
    if row['Action Points']:
        md += """\n**Action Points:** {Action Points}  """.format(**row_dict)
    
    if row['Fort']:
        md += """\n**AC:** {AC}; **Fort:** {Fort}, **Ref:** {Ref}, **Will:** {Will}""".format(**row_dict)
    else: 
        md += """\n**Defenses:** {AC}""".format(**row_dict) 
    
    if row['Saving Throws']:
        md += """; **Saving Throws:** +{Saving Throws}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 
    
    md += """\n**HP:** {HP}""".format(**row_dict)

    if row['Staggered']:
        md += """, **Staggered:** {Staggered}""".format(**row_dict)
    
    if row['HP Extras']:
        md += """; {HP Extras}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 
    
    if row['Immunity to']:
        md += """\n""".format(**row_dict)
    elif row['Resistance to']:
        md += """\n""".format(**row_dict) 
    elif row['Vulnerability to']:
        md += """\n""".format(**row_dict) 

    if row['Immunity to']:
        md += """**Immune:**&nbsp;{Immunity to}""".format(**row_dict)
    
    if row['Resistance to']:
        if row['Immunity to']:
            md += """, """.format(**row_dict)
            
    if row['Resistance to']:
        md += """**Resist:**&nbsp;{Resistance to}""".format(**row_dict)
 
    if row['Vulnerability to']:
        if row['Resistance to']:
            md += """, """.format(**row_dict)
        elif row['Immunity to']: 
            md += """, """.format(**row_dict)
    
    if row['Vulnerability to']:
        md += """**Vulnerable:**&nbsp;{Vulnerability to}""".format(**row_dict)
    
    md += """   """.format(**row_dict) 

    if row['Aura']:
        md += """\n{Aura}  """.format(**row_dict)
    
    md += """\n""".format(**row_dict) 
    
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 1 - Tags'):
        row_dict['Power 1 - Tags'] = '● **{Power 1 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 1 - Range'):
        row_dict['Power 1 - Range'] = '{Power 1 - Range}; '.format(**row_dict)
    
       
    # Need to untangle so Rider displayed regardless of Defense existing
    
    if row_dict.get('Power 1 - Type') == 'Melee':
        row_dict['Power 1 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 1 - Type') == 'Basic Melee':
        row_dict['Power 1 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 1 - Type') == 'Ranged':
        row_dict['Power 1 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 1 - Type') == 'Basic Ranged':
        row_dict['Power 1 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 1 - Type') == 'Near':
        row_dict['Power 1 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 1 - Type') == 'Far':
        row_dict['Power 1 - Type'] = '⋇	'.format(**row_dict)
        
#     if row_dict.get('Power 1 - Type') == 'Melee':
#         row_dict['Power 1 - Type'] = '⚔️ '.format(**row_dict)
 
#     if row_dict.get('Power 1 - Type') == 'Basic Melee':
#         row_dict['Power 1 - Type'] = '<span style="height: 25px; width: 25px; background-color: #fff; border: 1.25px solid #000; border-radius: 50%; display:inline-block;">⚔️</span> '.format(**row_dict)

#     if row_dict.get('Power 1 - Type') == 'Ranged':
#         row_dict['Power 1 - Type'] = '🏹 '.format(**row_dict)
 
#     if row_dict.get('Power 1 - Type') == 'Basic Ranged':
#         row_dict['Power 1 - Type'] = '<span style="height: 25px; width: 25px; background-color: #fff; border: 1.25px solid #000; border-radius: 50%; display:inline-block;">🏹</span> '.format(**row_dict)

#     if row_dict.get('Power 1 - Type') == 'Near':
#         row_dict['Power 1 - Type'] = '💥 '.format(**row_dict)

#     if row_dict.get('Power 1 - Type') == 'Far':
#         row_dict['Power 1 - Type'] = '💣 '.format(**row_dict)
        
    if row['Power 1 - Name']:
        md += """\n{Power 1 - Type}**{Power 1 - Name}**""".format(**row_dict)

    if row_dict.get('Power 1 - Frequency'):
        row_dict['Power 1 - Frequency'] = ', {Power 1 - Frequency}'.format(**row_dict)   
        
    if row['Power 1 - Action']:        
        md += """ ({Power 1 - Action}{Power 1 - Frequency})""".format(**row_dict)
        
    if row['Power 1 - Name']:
        md += """ {Power 1 - Tags}  """.format(**row_dict)
        md += """\n{Power 1 - Range}{Power 1 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 1 - Defense']:
        if row['Power 1 - Rider']:
            md += """ vs {Power 1 - Defense} ({Power 1 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 1 - Defense}; """.format(**row_dict)
    elif row['Power 1 - Rider']:
        md += """{Power 1 - Rider}; """.format(**row_dict)
    
    if row['Power 1 - Name']:
        md += """{Power 1 - Effect}  """.format(**row_dict)
    
    if row['Power 1 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 1 - Secondary Attack}  """.format(**row_dict)

        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 2 - Tags'):
        row_dict['Power 2 - Tags'] = '● **{Power 2 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 2 - Range'):
        row_dict['Power 2 - Range'] = '{Power 2 - Range}; '.format(**row_dict)
    

    if row_dict.get('Power 2 - Type') == 'Melee':
        row_dict['Power 2 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 2 - Type') == 'Basic Melee':
        row_dict['Power 2 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 2 - Type') == 'Ranged':
        row_dict['Power 2 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 2 - Type') == 'Basic Ranged':
        row_dict['Power 2 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 2 - Type') == 'Near':
        row_dict['Power 2 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 2 - Type') == 'Far':
        row_dict['Power 2 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 2 - Name']:
        md += """\n{Power 2 - Type}**{Power 2 - Name}**""".format(**row_dict)

    if row_dict.get('Power 2 - Frequency'):
        row_dict['Power 2 - Frequency'] = ', {Power 2 - Frequency}'.format(**row_dict)   
        
    if row['Power 2 - Action']:        
        md += """ ({Power 2 - Action}{Power 2 - Frequency})""".format(**row_dict)
        
    if row['Power 2 - Name']:
        md += """ {Power 2 - Tags}  """.format(**row_dict)
        md += """\n{Power 2 - Range}{Power 2 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 2 - Defense']:
        if row['Power 2 - Rider']:
            md += """ vs {Power 2 - Defense} ({Power 2 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 2 - Defense}; """.format(**row_dict)
    elif row['Power 2 - Rider']:
        md += """{Power 2 - Rider}; """.format(**row_dict)
    
    if row['Power 2 - Name']:
        md += """{Power 2 - Effect}  """.format(**row_dict)
    
    if row['Power 2 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 2 - Secondary Attack}  """.format(**row_dict)

        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 3 - Tags'):
        row_dict['Power 3 - Tags'] = '● **{Power 3 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 3 - Range'):
        row_dict['Power 3 - Range'] = '{Power 3 - Range}; '.format(**row_dict)
    
       
    if row_dict.get('Power 3 - Type') == 'Melee':
        row_dict['Power 3 - Type'] = '†	'.format(**row_dict)
    
    if row_dict.get('Power 3 - Type') == 'Basic Melee':
        row_dict['Power 3 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 3 - Type') == 'Ranged':
        row_dict['Power 3 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 3 - Type') == 'Basic Ranged':
        row_dict['Power 3 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 3 - Type') == 'Near':
        row_dict['Power 3 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 3 - Type') == 'Far':
        row_dict['Power 3 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 3 - Name']:
        md += """\n{Power 3 - Type}**{Power 3 - Name}**""".format(**row_dict)

    if row_dict.get('Power 3 - Frequency'):
        row_dict['Power 3 - Frequency'] = ', {Power 3 - Frequency}'.format(**row_dict)   
        
    if row['Power 3 - Action']:        
        md += """ ({Power 3 - Action}{Power 3 - Frequency})""".format(**row_dict)
        
    if row['Power 3 - Name']:
        md += """ {Power 3 - Tags}  """.format(**row_dict)
        md += """\n{Power 3 - Range}{Power 3 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 3 - Defense']:
        if row['Power 3 - Rider']:
            md += """ vs {Power 3 - Defense} ({Power 3 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 3 - Defense}; """.format(**row_dict)
    elif row['Power 3 - Rider']:
        md += """{Power 3 - Rider}; """.format(**row_dict)
    
    if row['Power 3 - Name']:
        md += """{Power 3 - Effect}  """.format(**row_dict)
        
    if row['Power 3 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 3 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 4 - Tags'):
        row_dict['Power 4 - Tags'] = '● **{Power 4 - Tags}**'.format(**row_dict)

    if row_dict.get('Power 4 - Range'):
        row_dict['Power 4 - Range'] = '{Power 4 - Range}; '.format(**row_dict)
    
       
    if row_dict.get('Power 4 - Type') == 'Melee':
        row_dict['Power 4 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 4 - Type') == 'Basic Melee':
        row_dict['Power 4 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 4 - Type') == 'Ranged':
        row_dict['Power 4 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 4 - Type') == 'Basic Ranged':
        row_dict['Power 4 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 4 - Type') == 'Near':
        row_dict['Power 4 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 4 - Type') == 'Far':
        row_dict['Power 4 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 4 - Name']:
        md += """\n{Power 4 - Type}**{Power 4 - Name}**""".format(**row_dict)

    if row_dict.get('Power 4 - Frequency'):
        row_dict['Power 4 - Frequency'] = ', {Power 4 - Frequency}'.format(**row_dict)   
        
    if row['Power 4 - Action']:        
        md += """ ({Power 4 - Action}{Power 4 - Frequency})""".format(**row_dict)
        
    if row['Power 4 - Name']:
        md += """ {Power 4 - Tags}  """.format(**row_dict)
        md += """\n{Power 4 - Range}{Power 4 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 4 - Defense']:
        if row['Power 4 - Rider']:
            md += """ vs {Power 4 - Defense} ({Power 4 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 4 - Defense}; """.format(**row_dict)
    elif row['Power 4 - Rider']:
        md += """{Power 4 - Rider}; """.format(**row_dict)
    
    if row['Power 4 - Name']:
        md += """{Power 4 - Effect}  """.format(**row_dict)
        
    if row['Power 4 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 4 - Secondary Attack}  """.format(**row_dict)
        
        #modify tags a little, since they aren't always present    
    if row_dict.get('Power 5 - Tags'):
        row_dict['Power 5 - Tags'] = '● **{Power 5 - Tags}**'.format(**row_dict)
        
    if row_dict.get('Power 5 - Range'):
        row_dict['Power 5 - Range'] = '{Power 5 - Range}; '.format(**row_dict)
    
       
    if row_dict.get('Power 5 - Type') == 'Melee':
        row_dict['Power 5 - Type'] = '†	'.format(**row_dict)
 
    if row_dict.get('Power 5 - Type') == 'Basic Melee':
        row_dict['Power 5 - Type'] = '‡	'.format(**row_dict)

    if row_dict.get('Power 5 - Type') == 'Ranged':
        row_dict['Power 5 - Type'] = '↗	'.format(**row_dict)
 
    if row_dict.get('Power 5 - Type') == 'Basic Ranged':
        row_dict['Power 5 - Type'] = '⤢	'.format(**row_dict)

    if row_dict.get('Power 5 - Type') == 'Near':
        row_dict['Power 5 - Type'] = '∢	'.format(**row_dict)

    if row_dict.get('Power 5 - Type') == 'Far':
        row_dict['Power 5 - Type'] = '⋇	'.format(**row_dict)
        
    if row['Power 5 - Name']:
        md += """\n{Power 5 - Type}**{Power 5 - Name}**""".format(**row_dict)

    if row_dict.get('Power 5 - Frequency'):
        row_dict['Power 5 - Frequency'] = ', {Power 5 - Frequency}'.format(**row_dict)   
        
    if row['Power 5 - Action']:        
        md += """ ({Power 5 - Action}{Power 5 - Frequency})""".format(**row_dict)
        
    if row['Power 5 - Name']:
        md += """ {Power 5 - Tags}  """.format(**row_dict)
        md += """\n{Power 5 - Range}{Power 5 - Attack Bonus}""".format(**row_dict)
    
    if row['Power 5 - Defense']:
        if row['Power 5 - Rider']:
            md += """ vs {Power 5 - Defense} ({Power 5 - Rider}); """.format(**row_dict)
        else: 
            md += """ vs {Power 5 - Defense}; """.format(**row_dict)
    elif row['Power 5 - Rider']:
        md += """{Power 5 - Rider}; """.format(**row_dict)
    
    if row['Power 5 - Name']:
        md += """{Power 5 - Effect}  """.format(**row_dict)
    
    if row['Power 5 - Secondary Attack']:
        md += """\n*Secondary Attack:* {Power 5 - Secondary Attack}  """.format(**row_dict)
        
 
    
    if row['Description']:
        md += """\n\n{Description}  """.format(**row_dict) 
            
    #finally we remove the empty tags
    return md.replace('+-','-')

In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = companion_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Companions.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))